In [ ]:
import fitz 
import os

def extract_text_from_pdfs(pdf_folder):
    documents = []
    
    for file in os.listdir(pdf_folder):
        if file.endswith(".pdf"):
            doc_path = os.path.join(pdf_folder, file)
            doc = fitz.open(doc_path)

            text = ""
            for page in doc:
                text += page.get_text("text") + "\n"
            
            documents.append({"filename": file, "content": text})
    
    return documents

# Exemplo de uso
pdf_folder = "C:/Users/Administrador/Downloads/Procurement-_AI_Agent/data"
pdf_data = extract_text_from_pdfs(pdf_folder)

# Exibir o primeiro artigo extraído
print(pdf_data[0]["content"][:1000])  # Mostra os primeiros 1000 caracteres


See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/326790494
ANÁLISE DE RISCO E CLASSIFICAÇÃO DE FORNECEDORES EM UMA CADEIA
DE SUPRIMENTOS
Article  in  Engevista · November 2012
DOI: 10.22409/engevista.v15i2.461
CITATIONS
3
READS
1,196
5 authors, including:
Miguel Afonso Sellitto
Universidade do Vale do Rio dos Sinos, Brazil
304 PUBLICATIONS   3,880 CITATIONS   
SEE PROFILE
Miriam Borchardt
Universidade do Vale do Rio dos Sinos
156 PUBLICATIONS   2,411 CITATIONS   
SEE PROFILE
Giancarlo Medeiros Pereira
Universidade do Vale do Rio dos Sinos
137 PUBLICATIONS   2,286 CITATIONS   
SEE PROFILE
All content following this page was uploaded by Miguel Afonso Sellitto on 23 August 2018.
The user has requested enhancement of the downloaded file.

 

 






	




	

	

	





















ANÁLISE DE RISCO E CLASSIFICAÇÃO DE FORNECEDORES EM UMA 
CADEIA DE SUPRIMENTOS
Miguel Afonso Sellitto1
Bruna Omizzo


In [2]:
len(pdf_data[0]["content"])

50571

In [3]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

# Criar um cliente do ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# Definir a função de embedding (modelo de NLP para transformar texto em vetor)
embedding_function = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

# Criar coleção
collection = chroma_client.get_or_create_collection(name="procurement_articles", embedding_function=embedding_function)

# Adicionar os artigos processados
for idx, doc in enumerate(pdf_data):
    collection.add(
        ids=[str(idx)], 
        documents=[doc["content"]],
        metadatas=[{"filename": doc["filename"]}]
    )

print("Artigos indexados com sucesso! ✅")

print(type(collection))

c:\Users\Administrador\Downloads\Procurement-_AI_Agent\ambiente_virtual\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Insert of existing embedding ID: 0
Add of existing embedding ID: 0
Insert of existing embedding ID: 1
Add of existing embedding ID: 1
Insert of existing embedding ID: 2
Add of existing embedding ID: 2
Insert of existing embedding ID: 3
Add of existing embedding ID: 3
Insert of existing embedding ID: 4
Add of existing embedding ID: 4
Insert of existing embedding ID: 5
Add of existing embedding ID: 5
Insert of existing embedding ID: 6
Add of existing embedding ID: 6
Insert of existing embedding ID: 7
Add of existing embedding ID: 7
Insert of existing embedding ID: 8
Add of existing embedding ID: 8
Insert of existing embedding ID: 9
Add of existing embedding ID: 9


Artigos indexados com sucesso! ✅
<class 'chromadb.api.models.Collection.Collection'>


In [ ]:
import ollama

def search_articles(query, top_k=3):
    results = collection.query(
        query_texts=[query],
        n_results=top_k
    )
    return results["documents"]

# Exemplo de busca
query = "Quais são os principais desafios estratégicos no procurement?"
context = search_articles(query)
print("\n".join(context[0]))  # Exibe o artigo mais relevante


def ask_llava_with_context(question):

    #gtp = "Voce é o Chat GTP, um agente especialista no assunto de procurement e compras globais. Seu papel é auxiliar a liderança na tomada de decisões estratégicas para a empresa. Não precisa se identificar toda vez"

    context = search_articles(question) # busca nos embbedings
    context_str = "\n\n".join(context[0])  # Concatenar o conteúdo dos artigos

    prompt = f"""
    Baseado nos seguintes artigos sobre procurement:
    {context_str}

    Responda à seguinte pergunta:
    {question}
    """

    response = ollama.chat(model="llava", messages=[{"role": "user", "content": prompt}])
    return response["message"]["content"]

# Exemplo de uso
question = "Como podemos melhorar a eficiência na negociação com fornecedores e eliminar atrasos com PN's que sofrem evolução de traço durante o processo?"
answer = ask_llava_with_context(question)
print(answer)


 Para melhorar a eficiência na negociação com fornecedores e eliminar atrasos com PN's que sofrem evolução de traço durante o processo, podemos seguir algumas abordagens:

1. Monitoramento constante do processo: é importante ter um monitoramento contínuo do processo para garantir que todas as etapas estão sendo executadas conforme o plano e que houve quaisquer alterações no traço dos PN's.
2. Definição de métricas: estabelecer métricas que medam a eficiência da negociação com fornecedores, como por exemplo, o tempo de resposta das solicitações ou o número de rejeições dos PN's.
3. Planejamento estratégico: implementar um planejamento estratégico para a negociação com fornecedores que tenha em conta as tendências do mercado e as alterações nos traços dos PN's. Isso pode incluir o desenvolvimento de relacionamentos com os fornecedores, análises de preço e comparação de custos para garantir que as negociações sejam mais favoráveis à empresa.
4. Treinamento contínuo: oferecer treinamento c

In [2]:
import os

def list_folders(root_folder, exclude_folder):
    for dirpath, dirnames, _ in os.walk(root_folder):
        dirnames[:] = [d for d in dirnames if d != exclude_folder]  # Ignorar o ambiente virtual
        for dirname in dirnames:
            print(os.path.join(dirpath, dirname))

# Exemplo de uso
root_directory = "C:/Users/Administrador/Downloads/Procurement-_AI_Agent"
exclude_directory = "ambiente_virtual"  # Nome do ambiente virtual
list_folders(root_directory, exclude_directory)


C:/Users/Administrador/Downloads/Procurement-_AI_Agent\.git
C:/Users/Administrador/Downloads/Procurement-_AI_Agent\AI_models
C:/Users/Administrador/Downloads/Procurement-_AI_Agent\backend
C:/Users/Administrador/Downloads/Procurement-_AI_Agent\data
C:/Users/Administrador/Downloads/Procurement-_AI_Agent\frontend
C:/Users/Administrador/Downloads/Procurement-_AI_Agent\.git\hooks
C:/Users/Administrador/Downloads/Procurement-_AI_Agent\.git\info
C:/Users/Administrador/Downloads/Procurement-_AI_Agent\.git\logs
C:/Users/Administrador/Downloads/Procurement-_AI_Agent\.git\objects
C:/Users/Administrador/Downloads/Procurement-_AI_Agent\.git\refs
C:/Users/Administrador/Downloads/Procurement-_AI_Agent\.git\logs\refs
C:/Users/Administrador/Downloads/Procurement-_AI_Agent\.git\logs\refs\heads
C:/Users/Administrador/Downloads/Procurement-_AI_Agent\.git\logs\refs\remotes
C:/Users/Administrador/Downloads/Procurement-_AI_Agent\.git\logs\refs\remotes\origin
C:/Users/Administrador/Downloads/Procurement-_AI_A